# 1. Gün bölüm 1: Basit kompartman modelinin gösterimi

<div style="background: #f8f9fa; padding: 0.5rem 1rem; border-radius: 8px; margin-bottom: 1rem; border-left: 4px solid #007bff;">
<a href="../../index-tr.ipynb" style="text-decoration: none; color: #007bff; font-weight: bold;">← Kurs Ana Sayfasına Dön</a> | 
<a href="../../index.ipynb" style="text-decoration: none; color: #dc3545; font-weight: bold;">🇬🇧 English Index</a>
</div>

---

Önceki oturumlarımızda kompartman modellerinin enfeksiyöz hastalıkları tanımlamak için nasıl kullanılabileceğini gözden geçirdik. Ayrıca geçiş oranlarının nasıl belirleneceğini, bunun zaman dağılımları ile ilişkisini ve basit modelleme çıktısını nasıl yorumlayacağımızı inceledik. Bu ilk uygulamada önce basit bir kohort modelinin nasıl kodlandığını göreceğiz ve ayrıca modelin temel varsayımlarını test etmeyi inceleyeceğiz.

## 1. Basit bir kohort modeli

Çalıştığımız kohort modelini hatırlayın, enfekte bireylerin başlangıç popülasyonu ile başlıyoruz ve iyileşmeye geçişe izin veriyoruz. Bu modelin temel yapı taşlarının neler olduğuna bakalım.

In [ ]:

# Bazı yararlı paketleri yükle:
library(deSolve)  # Diferansiyel denklemleri çözmek için 
library(reshape2) # Model çıktısını manipüle etmek için
library(ggplot2)  # Güzel grafikler üretmek için!
library(here)     # Her zaman çalışma dizinimizde olduğumuzdan emin olmak için


###################
#  Şimdi kohort modelimizi kodlayalım 

# Toplam popülasyonumuzu N olarak adlandıracağız. N için bir parametre oluşturalım
N <- 1000

### Bazı model girdileri

# Başlangıç koşulları: Kodlamak istediğimiz kohort modeli iki aşamaya sahip, I Enfekte ve R (iyileşen)

# Modelin durum değişkenlerini içeren bir vektör oluşturalım

initial_state_values <- c(
                          I = N,
                          R = 0)           

# I ve R model kompartmanlarına başlangıç değerleri atadığımıza dikkat edin

# Günlük parametre değerleri: Enfeksiyöz dönemin ortalama 2 gün olduğunu biliyoruz
# Buna göre iyileşme oranını (gamma) şu şekilde ifade edebiliriz:

parameters <- c(gamma = 1/2)


# ZAMAN ADIMLARI:

# Bu, modeli çözmek için zaman adımları dizisini saklayan bir vektördür
times <- seq(from = 0, to = 50, by = 1)   # 0'dan 50 güne kadar günlük aralıklarla


### Şimdi modeli kodla ! 

## Adi Diferansiyel Denklemler 
# Aşağıdaki model fonksiyonu girdi argümanları olarak 
# (şu sırayla): zaman, durum ve parametreleri alır
# Diferansiyel denklemlerin nasıl kodlandığına dikkat edin ve notlarınızla karşılaştırın ! 

cohort_model <- function(time, state, parameters) {  
  
  with(as.list(c(state, parameters)), {     
    
    # Toplam popülasyon büyüklüğü N'yi hesaplama (her kompartmandaki kişi sayısının toplamı)
    N <- I+R
    
    # Diferansiyel denklemler
    dI  <- -gamma * I     
    dR <- gamma * I

    return(list(c(dI, dR))) 
  })
  
}


### 1.1 Kohort modelini çalıştırma

I ve R kompartmanlı basit bir kohort modeli kodladık, bu durumlar için başlangıç koşulları belirledik ve ayrıca iyileşme oranını yansıtmak için geçiş parametresi gamma'yı tanımladık.

Şimdi modeli çalıştıralım ve bazı çıktıları görelim:

In [ ]:
## MODEL ÇIKTISI (Diferansiyel denklemleri ode entegrasyon algoritması kullanarak çözme)

# ode fonksiyonunu (deSolve paketinden) kullanıyoruz ve yukarıda oluşturduğumuz 
# sistem ve parametreleri geçiyoruz

# Model çıktısı output1 veri çerçevesinde saklanacak

output1 <- as.data.frame(ode(y = initial_state_values, 
                             times = times, 
                             func = cohort_model,
                             parms = parameters))

# Konsolda output1 yazarak çıktıya bir göz atalım

# Her durum değişkeninin (I ve R) her birinin bir sütunu olduğunu görüyorsunuz
# Bu sonuçları çizmek için uzun format dediğimiz şeye ihtiyacımız var, bu yüzden rehsape 
# paketini ve "melt" fonksiyonunu kullanarak çıktımızı o formatta alalım

# çıktı veri setini uzun formata çevir
output1_long <- melt(as.data.frame(output1), id = "time")                  

# Konsolda output1_long yazarak çıktıya bir göz atalım


## Model çıktısını çiz

# I ve R kompartmanlarındaki kişi sayısını zaman içinde çiz
ggplot(data = output1_long,                                              # çizilecek veriyi içeren nesneyi belirt
       aes(x = time, y = value, colour = variable, group = variable)) +  # sütunları eksenlere ve gruplara ata
  geom_line(size=2) +                                                    # veriyi çizgiler olarak temsil et
  xlab("Zaman (gün)")+                                                   # x ekseni için etiket ekle
  ylab("Sayı") +                                                         # y ekseni için etiket ekle
  labs(colour = "Kompartman",                                            # lejand başlığı ekle  
  title = "Kohort modeli")  





Görev: Yukarıdaki kodu kullanarak aynı modeli çalıştırmayı keşfedin, ancak şimdi kohortumuzda ortalama enfeksiyöz dönemin iki gün yerine 10 gün olduğu bir senaryoyu hayal edin. Grafiğin neden farklı göründüğünü açıklayabilir misiniz?